In [1]:
#!pip install bs4
#!pip install selenium
#!pip install webdriver-manager

In [1]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from webdriver_manager.microsoft import EdgeChromiumDriverManager

'''
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
#options.add_argument('--headless')  #ẩn cửa sổ chrome
options.add_argument("--window-size=1920,1080")
'''

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [2]:
service=EdgeService(EdgeChromiumDriverManager().install())

[WDM] - Downloading: 100%|██████████| 7.86M/7.86M [00:05<00:00, 1.38MB/s]


In [3]:
driver = webdriver.Edge(service=service)

In [4]:
url = 'https://charts.spotify.com/'

username = 'tungbtt.2002@gmail.com' 
password = 'temp_@cc'

In [5]:
driver.get(url)
driver.implicitly_wait(5)

driver.find_element(By.XPATH, '//*[@id="__next"]/div/div/main/div[2]/div/header/div/div[2]/a/div[1]').click()
driver.implicitly_wait(5)

driver.find_element(By.XPATH, '//*[@id="login-username"]').send_keys(username)
driver.find_element(By.XPATH, '//*[@id="login-password"]').send_keys(password)

driver.implicitly_wait(2)
driver.find_element(By.XPATH, '//*[@id="login-button"]/span[1]').click()

In [7]:
#driver.save_screenshot('test.png')

## **Lấy dữ liệu trong khoảng thời gian tùy ý:**

Lưu ý nhớ kiểm tra `end_date` có tồn tại  hay không.

In [26]:
import pandas as pd
import numpy as np

In [18]:
import requests

CLIENT_ID = '29942d75c20b46b9b01eb4cd31edff20'
CLIENT_SECRET = 'e8905b5241e54a6b8340e94282abb615'

In [37]:
AUTH_URL = 'https://accounts.spotify.com/api/token'
BASE_URL = 'https://api.spotify.com/v1/'

In [42]:
import os
from datetime import date
from datetime import datetime, timedelta

def generate_date_list(start_date,end_date):
    current_date = start_date
    date_list = []

    while current_date <= end_date:
        formatted_date = current_date.strftime('%Y-%m-%d')
        date_list.append(formatted_date)
        current_date += timedelta(days=1)

    return date_list


start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 9, 30)
date_list = generate_date_list(start_date,end_date)
print(date_list)

['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06', '2023-01-07', '2023-01-08', '2023-01-09', '2023-01-10', '2023-01-11', '2023-01-12', '2023-01-13', '2023-01-14', '2023-01-15', '2023-01-16', '2023-01-17', '2023-01-18', '2023-01-19', '2023-01-20', '2023-01-21', '2023-01-22', '2023-01-23', '2023-01-24', '2023-01-25', '2023-01-26', '2023-01-27', '2023-01-28', '2023-01-29', '2023-01-30', '2023-01-31', '2023-02-01', '2023-02-02', '2023-02-03', '2023-02-04', '2023-02-05', '2023-02-06', '2023-02-07', '2023-02-08', '2023-02-09', '2023-02-10', '2023-02-11', '2023-02-12', '2023-02-13', '2023-02-14', '2023-02-15', '2023-02-16', '2023-02-17', '2023-02-18', '2023-02-19', '2023-02-20', '2023-02-21', '2023-02-22', '2023-02-23', '2023-02-24', '2023-02-25', '2023-02-26', '2023-02-27', '2023-02-28', '2023-03-01', '2023-03-02', '2023-03-03', '2023-03-04', '2023-03-05', '2023-03-06', '2023-03-07', '2023-03-08', '2023-03-09', '2023-03-10', '2023-03-11', '2023-03-12', '2023

In [43]:
len(date_list)

273

In [54]:
for d in date_list:
    
    print(d)
    print("The Spotify Global Chart is being crawled...")
    driver.get(f"https://charts.spotify.com/charts/view/regional-global-daily/{d}")
    
    table = driver.find_element(By.CSS_SELECTOR, 'table').find_elements(By.TAG_NAME, 'tbody')[0]
    rows = table.find_elements(By.TAG_NAME, 'tr')


        
    song_data_list = []            

    for row in rows:

        # lấy id
        link_element = row.find_element(By.XPATH, './td[3]/div/div[2]/a') 
        link_href = link_element.get_attribute('href') # 
        
        
        # lấy dữ liệu của dòng
        elements = row.find_elements(By.TAG_NAME, 'td')
        
        song_data = {
            'date' : d,
            'position' : int(elements[1].text.split('\n')[0]),
            'id' : link_href.replace('https://open.spotify.com/track/',''),
            'track' : elements[2].text.split('\n')[0],
            'artist' : elements[2].text.split('\n')[1],
            'peak' : int(elements[3].text),
            'prev' : int(elements[4].text) if elements[4].text != '—' else -1, # entry hoặc new sẽ mang giá trị -1
            'streak' : int(elements[5].text),
            'streams': int(elements[6].text.replace(',', ''))
        }
        
        song_data_list.append(song_data)

    global_chart = pd.DataFrame(song_data_list)    
    print(len(global_chart))
    
    
    header = not os.path.exists('global_chart_2023.csv') or os.stat('global_chart_2023.csv').st_size == 0
    global_chart.to_csv('global_chart_2023.csv', mode='a', header=header, index=False)
    
    #------------------------------------------------------------
    print("The Spotify Vietnam Chart is being crawled...")
    driver.get(f"https://charts.spotify.com/charts/view/regional-vn-daily/{d}")
    
    table = driver.find_element(By.CSS_SELECTOR, 'table').find_elements(By.TAG_NAME, 'tbody')[0]

    rows = table.find_elements(By.TAG_NAME, 'tr')
        
    song_data_list = []            

    for row in rows:

        link_element = row.find_element(By.XPATH, './td[3]/div/div[2]/a')
        link_href = link_element.get_attribute('href')
        
        elements = row.find_elements(By.TAG_NAME, 'td')
        
        song_data = {
            'date' : d,
            'position' : int(elements[1].text.split('\n')[0]),
            'id' : link_href.replace('https://open.spotify.com/track/',''),
            'track' : elements[2].text.split('\n')[0],
            'artist' : elements[2].text.split('\n')[1],
            'peak' : int(elements[3].text),
            'prev' : int(elements[4].text) if elements[4].text != '—' else -1, # entry hoặc new sẽ mang giá trị -1
            'streak' : int(elements[5].text),
            'streams': int(elements[6].text.replace(',', ''))
        }
        
        song_data_list.append(song_data)


    vn_chart = pd.DataFrame(song_data_list)
    print(len(vn_chart))
    header = not os.path.exists('vn_chart_2023.csv') or os.stat('vn_chart_2023.csv').st_size == 0
    vn_chart.to_csv('vn_chart_2023.csv', mode='a', header=header, index=False)
    
    
    #-------------------------------------
    check = not os.path.exists('audio_features_2023.csv') or os.stat('audio_features_2023.csv').st_size == 0

    if check:
        exist_tracks = set()
    else:
        exist_tracks = set(pd.read_csv('audio_features_2023.csv')['id'])
    
    
    vn_chart_ids = set(vn_chart['id'])
    global_chart_ids = set(global_chart['id'])
    not_exist_tracks = vn_chart_ids.union(global_chart_ids).difference(exist_tracks) # chỉ thu thập từ những bài chưa có
    
    if not_exist_tracks: 
        auth_response = requests.post(AUTH_URL, {
            'grant_type': 'client_credentials',
            'client_id': CLIENT_ID,
            'client_secret': CLIENT_SECRET,
        })
    
        headers = {
        'Authorization': 'Bearer {token}'.format(token=access_token)
        }
        # convert the response to JSON
        auth_response_data = auth_response.json()
        # save the access token
        access_token = auth_response_data['access_token']
    
    audio_features_data = []
    print("The audio features are being crawled...")
    
    for track_id in not_exist_tracks:
        r = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers).json()
        in4 = requests.get(BASE_URL + 'tracks/' + track_id, headers=headers).json()
    
        r['explicit'] = in4['explicit']
        r['popularity'] = in4['popularity']
        r['release_date'] = in4['album']['release_date']
        r['album_id'] = in4['album']['id']
        r['album_name'] = in4['album']['name']
        r['track_number'] = in4['track_number']
        r['preview_url'] = in4['preview_url']
    
        audio_features_data.append(r)
    
    audio_features = pd.DataFrame(audio_features_data)
    print(len(audio_features))
    
    if len(audio_features) != 0:
        audio_features = audio_features.set_index('id').reset_index().drop(columns=['type','uri','track_href','analysis_url'])
        audio_features.to_csv('audio_features_2023.csv', mode='a', header=check, index=False)
    

2023-01-01
The Spotify Global Chart is being crawled...
199
The Spotify Vietnam Chart is being crawled...
200
The audio features are being crawled...
212
2023-01-02
The Spotify Global Chart is being crawled...
199
The Spotify Vietnam Chart is being crawled...
200
The audio features are being crawled...
30
2023-01-03
The Spotify Global Chart is being crawled...
199
The Spotify Vietnam Chart is being crawled...
200
The audio features are being crawled...
4
2023-01-04
The Spotify Global Chart is being crawled...
199
The Spotify Vietnam Chart is being crawled...
200
The audio features are being crawled...
2
2023-01-05
The Spotify Global Chart is being crawled...
199
The Spotify Vietnam Chart is being crawled...
200
The audio features are being crawled...
4
2023-01-06
The Spotify Global Chart is being crawled...
199
The Spotify Vietnam Chart is being crawled...
200
The audio features are being crawled...
4
2023-01-07
The Spotify Global Chart is being crawled...
199
The Spotify Vietnam Chart

### **Lấy dữ liệu của ngày mới nhất**

In [11]:
from datetime import datetime

def convert_to_desired_format(date_string):

    month, day = date_string.split(' ')
    month_number = datetime.strptime(month, '%B').month
    current_year = datetime.now().year
    
    input_date = datetime(current_year, month_number, int(day))
    if input_date < datetime.now():
        year = current_year
    else:
        year = current_year - 1
        
    formatted_date = f"{year}-{'{:02d}'.format(month_number)}-{day}"
    return formatted_date

In [12]:
date_picker_element = driver.find_element(By.XPATH, '//*[@id="date_picker"]')
date = convert_to_desired_format(date_picker_element.get_attribute('value'))

table = driver.find_element(By.CSS_SELECTOR, 'table').find_elements(By.TAG_NAME, 'tbody')[0]
rows = table.find_elements(By.TAG_NAME, 'tr')

#table = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div/main/div[2]/div[3]/div/table')
        
song_data_list = []            

for row in rows:

        link_element = row.find_element(By.XPATH, './td[3]/div/div[2]/a')
        link_href = link_element.get_attribute('href')
        
        elements = row.find_elements(By.TAG_NAME, 'td')
        
        song_data = {
            'date' : date,
            'position' : int(elements[1].text.split('\n')[0]),
            'id' : link_href.replace('https://open.spotify.com/track/',''),
            'track' : elements[2].text.split('\n')[0],
            'artist' : elements[2].text.split('\n')[1],
            'peak' : int(elements[3].text),
            'prev' : int(elements[4].text) if elements[4].text != '—' else -1,
            'streak' : int(elements[5].text),
            'streams': int(elements[6].text.replace(',', ''))
        }
        
        song_data_list.append(song_data)


global_chart = pd.DataFrame(song_data_list)

In [13]:
display(global_chart)

,date,position,id,track,artist,peak,prev,streak,streams
0,2023-09-29,1,56y1jOTK0XSvJzVv9vHQBK,Paint The Town Red,Doja Cat,1,1,57,7406155
1,2023-09-29,2,7x9aauaA9cu6tyfpHnqDLo,Seven (feat. Latto) (Explicit Ver.),"Jung Kook, Latto",1,2,78,7403939
2,2023-09-29,3,6ehWdR7cGDXnT7aKEASJxE,3D (feat. Jack Harlow),"Jung Kook, Jack Harlow",3,-1,1,6328084
3,2023-09-29,4,3rUGC1vUpkDG9CZFHMur1t,greedy,Tate McRae,4,4,15,4966154
4,2023-09-29,5,1BxfuPKGuaTgP7aM0Bbdwr,Cruel Summer,Taylor Swift,2,3,196,4804613
...,...,...,...,...,...,...,...,...,...
195,2023-09-29,196,2J2Z1SkXYghSajLibnQHOa,Unstoppable,Sia,89,189,3,1143887
196,2023-09-29,197,4PA1wK0leCjmRZlP5dQ8Lv,El Cielo,"Sky Rompiendo, Feid, Myke Towers",15,186,3,1143792
197,2023-09-29,198,2yPoXCs7BSIUrucMdK5PzV,Umbrella,"Rihanna, JAY-Z",15,-1,1,1137550
198,2023-09-29,199,0PZ04XgQ14FmKZBg3Gxlhr,Radio,Lana Del Rey,108,175,13,1137490


In [14]:
import os

In [15]:
header = not os.path.exists('global_chart.csv') or os.stat('global_chart.csv').st_size == 0
global_chart.to_csv('global_chart.csv', mode='a', header=header, index=False)

In [16]:
driver.get("https://charts.spotify.com/charts/view/regional-vn-daily/latest")

In [17]:
date_picker_element = driver.find_element(By.XPATH, '//*[@id="date_picker"]')
date = convert_to_desired_format(date_picker_element.get_attribute('value'))

table = driver.find_element(By.CSS_SELECTOR, 'table').find_elements(By.TAG_NAME, 'tbody')[0]

rows = table.find_elements(By.TAG_NAME, 'tr')
        
song_data_list = []            

for row in rows:

        link_element = row.find_element(By.XPATH, './td[3]/div/div[2]/a')
        link_href = link_element.get_attribute('href')
        
        elements = row.find_elements(By.TAG_NAME, 'td')
        
        song_data = {
            'date' : date,
            'position' : int(elements[1].text.split('\n')[0]),
            'id' : link_href.replace('https://open.spotify.com/track/',''),
            'track' : elements[2].text.split('\n')[0],
            'artist' : elements[2].text.split('\n')[1],
            'peak' : int(elements[3].text),
            'prev' : int(elements[4].text) if elements[4].text != '—' else -1,
            'streak' : int(elements[5].text),
            'streams': int(elements[6].text.replace(',', ''))
        }
        
        song_data_list.append(song_data)


vn_chart = pd.DataFrame(song_data_list)

In [18]:
display(vn_chart)

,date,position,id,track,artist,peak,prev,streak,streams
0,2023-09-29,1,7x9aauaA9cu6tyfpHnqDLo,Seven (feat. Latto) (Explicit Ver.),"Jung Kook, Latto",1,1,78,561898
1,2023-09-29,2,6ehWdR7cGDXnT7aKEASJxE,3D (feat. Jack Harlow),"Jung Kook, Jack Harlow",2,-1,1,327909
2,2023-09-29,3,3Ua0m0YmEjrMi9XErKcNiR,Like Crazy,Jimin,1,3,189,235158
3,2023-09-29,4,2pLZ6tUBapqlngc9lnYz4M,3D (feat. Jack Harlow) (Alternate Ver.),"Jung Kook, Jack Harlow",4,-1,1,224172
4,2023-09-29,5,5h1BN75CEh8wdSwE1xrbSe,Slow Dancing,V,2,2,15,194483
...,...,...,...,...,...,...,...,...,...
195,2023-09-29,196,64qXPpH0c7GkJOQcUj79Ia,Cô Đơn Trên Sofa,Hồ Ngọc Hà,46,197,2,13325
196,2023-09-29,197,2QyOYFAxW2khNYee4yeT6r,Thiêu Thân,"B Ray, Sofia, Châu Đăng Khoa, Masew",13,184,8,13229
197,2023-09-29,198,5cF0dROlMOK5uNZtivgu50,Attention,Charlie Puth,10,192,3,13120
198,2023-09-29,199,2g68ePl7VMltuOiuSTAz7I,Để Anh Một Mình (feat. Quang Anh Rhyder),"RAP VIỆT, Quang Anh Rhyder",48,190,118,13010


In [19]:
driver.quit()

In [20]:
header = not os.path.exists('vn_chart.csv') or os.stat('vn_chart.csv').st_size == 0
vn_chart.to_csv('vn_chart.csv', mode='a', header=header, index=False)

In [21]:
import requests

CLIENT_ID = '29942d75c20b46b9b01eb4cd31edff20'
CLIENT_SECRET = 'e8905b5241e54a6b8340e94282abb615'

In [22]:
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [23]:
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

BASE_URL = 'https://api.spotify.com/v1/'

In [25]:
check = not os.path.exists('audio_features.csv') or os.stat('audio_features.csv').st_size == 0

if check:
    exist_tracks = set()
else:
    exist_tracks = set(pd.read_csv('audio_features.csv')['id'])

In [26]:
vn_chart_ids = set(vn_chart['id'])
global_chart_ids = set(global_chart['id'])

not_exist_tracks = vn_chart_ids.union(global_chart_ids).difference(exist_tracks)

In [57]:
audio_features_data = []

for track_id in not_exist_tracks:
    r = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers).json()
    in4 = requests.get(BASE_URL + 'tracks/' + track_id, headers=headers).json()
    
    r['explicit'] = in4['explicit']
    r['popularity'] = in4['popularity']
    r['release_date'] = in4['album']['release_date']
    r['album_id'] = in4['album']['id']
    r['album_name'] = in4['album']['name']
    r['track_number'] = in4['track_number']
    r['preview_url'] = in4['preview_url']
    
    audio_features_data.append(r)
    
audio_features = pd.DataFrame(audio_features_data)

In [28]:
if len(audio_features) != 0:
    audio_features = audio_features.set_index('id').reset_index().drop(columns=['type','uri','track_href','analysis_url'])
    audio_features.to_csv('audio_features.csv', mode='a', header=check, index=False)

In [29]:
audio_features

""
